In [4]:
from rbc_parser import rbc_parser
from datetime import datetime, timedelta

# Задаем параметры запросы и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2021-01-01'
# dateTo = "2021-02-28"
dateTo = "2021-01-05"
offset = 0
limit = 100

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 100 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
# tbl = parser._get_search_table(param_dict,
#                                includeText = True) # Парсить текст статей
tbl = parser.get_articles2(param_dict,
                          count=10) # Парсить текст статей
print(len(tbl))
tbl.head()
len(tbl), tbl['text'][0][:100]

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.01.2021', 'dateTo': '05.01.2021', 'offset': '0', 'limit': '100', 'material': ''}
Parsing articles from  0
https://www.rbc.ru/v10/search/ajax/?project=rbcnews&category=TopRbcRu_economics&dateFrom=01.01.2021&dateTo=05.01.2021&offset=0&limit=100&query=РБК&material=


/home/misha-sh/rbc_parser.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(new_arcticles, ignore_index=True)


Parsing articles from  10
https://www.rbc.ru/v10/search/ajax/?project=rbcnews&category=TopRbcRu_economics&dateFrom=01.01.2021&dateTo=05.01.2021&offset=10&limit=100&query=РБК&material=
Finish
20


/home/misha-sh/rbc_parser.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(new_arcticles, ignore_index=True)


(20,
 'Минюст признал писателя Дмитрия Глуховского, автора романов «Метро 2033» и «Текст», и рэпера Oxxxymi')

In [92]:
# tokenize stopwords lemmatizer/stemmer(snowball)

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import pymorphy2
import string

ru_stopwords = set(stopwords.words("russian") + list(string.punctuation + '»' + '—' +  '«'))
only_words_tokenizer = RegexpTokenizer(r'\w+')
snowball = SnowballStemmer(language="russian")
morph = pymorphy2.MorphAnalyzer()

def tokenize(text):
    # text = only_words_tokenizer.tokenize(text) 
    text = word_tokenize(text, language="russian")
    res = []
    for word in text:
        if word not in ru_stopwords and len(word) > 1:
            res.append(word)
    return res

def keywords_calc(text):
    words = tokenize(text)
    res = {}
    for word in words:
        short = snowball.stem(word)
        if short not in res:
            normal = morph.parse(word)
            if len(normal) == 0:
                continue
            normal = normal[0]
            if normal.tag.POS == "INFN" or normal.tag.POS == "VERB" or normal.tag.POS == "ADJF":
                continue
            
            res[short] = { 
                "word": word, #normal.normal_form if normal.score > 0.5 else word,
                "short": short,
                "count": 0
            }
        res[short]["count"] += 1
            
    return res

# keywords_calc(tbl['text'][0][:200])
keywords_calc("ходить думать тест сообщает")

[nltk_data] Downloading package stopwords to /home/misha-
[nltk_data]     sh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/misha-sh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'тест': {'word': 'тест', 'short': 'тест', 'count': 1}}

In [86]:
import math
def idf_precalc(keyword_groups_list):
    from collections import defaultdict
    df_counts = defaultdict(lambda: 0)
    documents_count = 0
    for documents in keyword_groups_list:
        for document in documents:
            documents_count += 1
            for key in document:
                df_counts[key] += 1
    
    idf_counts = {}
    for word in df_counts:
        idf_counts[word] = math.log2(documents_count / df_counts[word])
        # idf_counts[word] = math.log2(math.log2(documents_count / df_counts[word]))
            
    return idf_counts

def tf_idf(documents, idfs):
    res = []
    for document in documents:
        total_words_count = 0
        for key in document:
            total_words_count += document[key]["count"]
        
        rres = {}
        for key in document:
            rres[key] = document[key].copy()
            rres[key]["count"] = 1 * idfs[key]
            # rres[key]["count"] = rres[key]["count"] / total_words_count * idfs[key]
            
        res.append(rres)
    return res

# idfs = idf_precalc([keyword_groups_before, keyword_groups_after])
# most_popular_keywords(keywords_sum(tf_idf(keyword_groups_before, idfs)), 10)

In [87]:
def keywords_sum(documents):
    res = {}
    for document in documents:
        for key in document:
            if key not in res:
                res[key] = document[key].copy()
            else:
                res[key]["count"] += document[key]["count"]
    return res


def keywords_norm(documents):
    rres = []
    for document in documents:
        res = {}
        for key in document:
            res[key] = document[key].copy()
            res[key]["count"] = 1
        rres.append(res)
        
    return rres

def keywords_mean(documents):
    res = keywords_sum(documents)
    for key in res:
        res[key]["count"] /= len(documents)
    return res

def most_popular_keywords(document, count):
    res = []
    for key in document:
        res.append(document[key])
    res.sort(key=lambda k: -k["count"])
    return res[:count]


# left - right
def keywords_diff(left, right):
    res = left.copy()
    for key in right:
        if key not in res:
            res[key] = right[key] 
            res[key]["count"] = -res[key]["count"]
        else:
            res[key]["count"] -= right[key]["count"]
    return res


In [88]:

most_popular_keywords(keywords_diff(keyword_groups_before[1], 
                                    keywords_mean(keyword_groups_before[2:])), 20)

[{'word': 'несерьезный', 'short': 'несерьезн', 'count': 9.861086905995395},
 {'word': 'АБ-4-19/12938',
  'short': 'аб-4-19/12938',
  'count': 9.861086905995395},
 {'word': '09.03.2021', 'short': '09.03.2021', 'count': 8.861086905995395},
 {'word': 'АБ-4-19/2990', 'short': 'аб-4-19/2990', 'count': 8.861086905995395},
 {'word': '5.1', 'short': '5.1', 'count': 8.84143926318831},
 {'word': '29.09.2022', 'short': '29.09.2022', 'count': 8.84143926318831},
 {'word': 'сверки', 'short': 'сверк', 'count': 8.276124405274238},
 {'word': 'Верховный', 'short': 'верховн', 'count': 7.5224422727241995},
 {'word': 'п.1', 'short': 'п.1', 'count': 7.505725734340368},
 {'word': 'нормальный', 'short': 'нормальн', 'count': 7.2438577781998506},
 {'word': 'выпустила', 'short': 'выпуст', 'count': 7.2438577781998506},
 {'word': '32', 'short': '32', 'count': 7.038091779982274},
 {'word': 'пп.', 'short': 'пп.', 'count': 6.769808587955543},
 {'word': 'нормативный', 'short': 'нормативн', 'count': 6.466662593689982},

In [93]:
from joblib import Parallel, delayed


def keywords_groups_calc(data):
    keywords_groups = Parallel(n_jobs=16)(delayed(keywords_calc)(i['text']) for i in data)
    return keywords_groups
    
keyword_groups_before = keywords_groups_calc(data_before)
keyword_groups_after = keywords_groups_calc(data_after)

idfs = idf_precalc([keyword_groups_before, keyword_groups_after])

keyword_groups_before = tf_idf(keyword_groups_before, idfs)
keyword_groups_after = tf_idf(keyword_groups_after, idfs)

most_popular_keywords(keywords_diff(keywords_mean(keyword_groups_before), 
                                    keywords_mean(keyword_groups_after)), 20)

[{'word': 'мобилизации', 'short': 'мобилизац', 'count': 0.4951839524167462},
 {'word': 'мобилизованные',
  'short': 'мобилизова',
  'count': 0.3922006618111935},
 {'word': 'Минцифры', 'short': 'минцифр', 'count': 0.29546572090821843},
 {'word': 'отсрочку', 'short': 'отсрочк', 'count': 0.29110744037655506},
 {'word': 'службе', 'short': 'служб', 'count': 0.27268124329042775},
 {'word': 'повестку', 'short': 'повестк', 'count': 0.26796184758568603},
 {'word': 'военкомат', 'short': 'военкомат', 'count': 0.260389442990471},
 {'word': 'Объясняем.рф',
  'short': 'объясняем.рф',
  'count': 0.2535191151160029},
 {'word': 'призыва', 'short': 'призыв', 'count': 0.2214796134705838},
 {'word': 'IT-компаний', 'short': 'IT-компан', 'count': 0.22071425698529773},
 {'word': 'Сотрудники', 'short': 'сотрудник', 'count': 0.20885739510032758},
 {'word': 'семей', 'short': 'сем', 'count': 0.2058571480280455},
 {'word': 'Минобороны', 'short': 'миноборон', 'count': 0.197158818224731},
 {'word': 'опубликован', '

In [94]:
keyword_groups_before = keywords_groups_calc(data_before)
keyword_groups_after = keywords_groups_calc(data_after)

# keywords = keywords_sum(keyword_groups)
# most_popular_keywords(keywords, 5)

# most_popular_keywords(keywords_diff(keyword_groups[0], keywords_mean(keyword_groups[1:])), 5)
# most_popular_keywords(keywords_diff(keyword_groups[0], keywords_mean(keyword_groups[60:])), 5)

most_popular_keywords(keywords_diff(keywords_mean(keywords_norm(keyword_groups_before)), 
                                    keywords_mean(keywords_norm(keyword_groups_after))), 20)

[{'word': 'мобилизации', 'short': 'мобилизац', 'count': 0.15873214211337416},
 {'word': 'мобилизованные',
  'short': 'мобилизова',
  'count': 0.10740416788148888},
 {'word': 'службе', 'short': 'служб', 'count': 0.0702097824426951},
 {'word': 'Сотрудники', 'short': 'сотрудник', 'count': 0.06868257736682078},
 {'word': 'отсрочку', 'short': 'отсрочк', 'count': 0.0671137212434226},
 {'word': 'Минцифры', 'short': 'минцифр', 'count': 0.06457300734446804},
 {'word': 'повестку', 'short': 'повестк', 'count': 0.055631915809349265},
 {'word': 'портале', 'short': 'портал', 'count': 0.053882571813347775},
 {'word': 'опубликован', 'short': 'опубликова', 'count': 0.05100864953420245},
 {'word': 'военкомат', 'short': 'военкомат', 'count': 0.050994765851694504},
 {'word': 'семей', 'short': 'сем', 'count': 0.04780151887486636},
 {'word': 'Объясняем.рф',
  'short': 'объясняем.рф',
  'count': 0.046357615894039736},
 {'word': 'призыва', 'short': 'призыв', 'count': 0.04470545767559387},
 {'word': 'октябрь',

In [95]:
keyword_groups_before = keywords_groups_calc(data_before)
keyword_groups_after = keywords_groups_calc(data_after)

most_popular_keywords(keywords_diff(keywords_mean(keyword_groups_before), 
                                    keywords_mean(keyword_groups_after)), 20)



[{'word': 'мобилизации', 'short': 'мобилизац', 'count': 0.24858733530481625},
 {'word': 'мобилизованные',
  'short': 'мобилизова',
  'count': 0.2124897607841504},
 {'word': 'Сотрудники', 'short': 'сотрудник', 'count': 0.12521693253918667},
 {'word': 'службе', 'short': 'служб', 'count': 0.1125966651394616},
 {'word': 'отсрочку', 'short': 'отсрочк', 'count': 0.10762630680161606},
 {'word': 'Минцифры', 'short': 'минцифр', 'count': 0.10662668166104378},
 {'word': 'повестку', 'short': 'повестк', 'count': 0.10430810668221639},
 {'word': 'РФ', 'short': 'рф', 'count': 0.07951184972302053},
 {'word': 'семей', 'short': 'сем', 'count': 0.07694336845905009},
 {'word': 'заявление', 'short': 'заявлен', 'count': 0.07133436072583892},
 {'word': 'лет', 'short': 'лет', 'count': 0.0674885806711372},
 {'word': 'военкомат', 'short': 'военкомат', 'count': 0.06644730448304108},
 {'word': 'учета', 'short': 'учет', 'count': 0.058297582850875364},
 {'word': 'призыва', 'short': 'призыв', 'count': 0.0579504907881

In [3]:
from cfo_parser import get_cfo_days
from parsers.kelrk_parser import parse_klerk_news

# data = get_cfo_days(30)
data = parse_klerk_news(30)
len(data)
for i in data:
    i["timestap"] /= 1000

930

In [49]:
import time
import datetime
time_begin = time.time() - datetime.timedelta(days=15).total_seconds()
data_before = list(filter(lambda i: time_begin <= i["timestap"], data))
data_after = list(filter(lambda i: time_begin > i["timestap"], data))

len(data_before), len(data_after)

(453, 477)

In [160]:
time_begin = time.time() - datetime.timedelta(days=15).total_seconds()
data_news = list(filter(lambda i: time_begin > i["timestap"], data))

keyword_groups = keywords_groups_calc(data_news)
idfs = idf_precalc([keyword_groups_before, keyword_groups_after])

keyword_groups = tf_idf(keyword_groups, idfs)
mmean = keywords_mean(keyword_groups)

popular_keywords = []
for document in keyword_groups:
    p = most_popular_keywords(keywords_diff(document, mmean), 50)
    popular_keywords.append({
        "short": set([i['short'] for i in p]), 
        "long": {
            i['short']: i for i in p
        }
    })
# popular_keywords[:3]

In [173]:
from tqdm.notebook import tqdm

def dist(d1, d2):
    s = 0
    for w in d1["short"] & d2["short"]:
        s += d1["long"][w]["count"] * d2["long"][w]["count"]
    return max(200 - s, 0)



distances = []
for i in tqdm(range(len(popular_keywords))):
    document = popular_keywords[i]
    d = []
    for j in range(len(popular_keywords)):
        if i == j:
            d.append(0)
        elif i > j:
            d.append(distances[j][i])
        else:
            document2 = popular_keywords[j]
            d.append(dist(document, document2))
    distances.append(d)
    
from sklearn.cluster import DBSCAN
import numpy as np
distances = np.array(distances)
clustering = DBSCAN(eps=3, min_samples=2, metric='precomputed').fit(distances)
import collections
counter = collections.Counter(clustering.labels_)
counter

  0%|          | 0/477 [00:00<?, ?it/s]

Counter({-1: 233,
         0: 2,
         1: 3,
         2: 79,
         3: 3,
         4: 12,
         5: 17,
         6: 2,
         7: 3,
         8: 7,
         9: 2,
         10: 2,
         11: 13,
         12: 2,
         13: 14,
         14: 2,
         15: 2,
         16: 4,
         17: 6,
         18: 5,
         19: 2,
         20: 2,
         21: 2,
         22: 2,
         23: 2,
         24: 2,
         25: 3,
         26: 2,
         27: 2,
         28: 3,
         29: 5,
         30: 2,
         31: 2,
         32: 3,
         33: 2,
         34: 3,
         35: 2,
         36: 4,
         37: 5,
         38: 3,
         39: 3,
         40: 6,
         41: 2})

In [200]:
def dist_most_common(d):
    s = 0
    from collections import defaultdict
    words = defaultdict(lambda:0)
    for i in range(len(d)):
        for j in range(i + 1, len(d)):
            d1 = d[i]
            d2 = d[j]
            for w in d1["short"] & d2["short"]:
                words[w] += d1["long"][w]["count"] * d2["long"][w]["count"]
                
    words = list(words.items())
    words.sort(key=lambda w: -w[1])
    return words[:10]

for label, count in counter.most_common(6):
    if label == -1:
        continue
    print("cluster: ", label, " count=", count)
    ddocs = []
    ddocs_i = []
    dpopular_keywords = []
    for i in range(len(keyword_groups)):
        if clustering.labels_[i] == label:
            ddocs.append(keyword_groups[i])
            ddocs_i.append(i)
            dpopular_keywords.append(popular_keywords[i])
            
    distances_i = []
    for i in ddocs_i:
        d = 0
        for j in ddocs_i:
            d += distances[i][j]
        distances_i.append((d, i))
    distances_i.sort(key=lambda i: i[0])     
    for score, i in distances_i[:3]:
        art = data_news[i]
        print(art["title"], art["url"])
    
    display(dict(dist_most_common(dpopular_keywords)))
    display(most_popular_keywords(keywords_mean(ddocs), 10))
    
    print()

cluster:  2  count= 79
💣 Ставку НДФЛ для работающих за границей дистанционщиков могут снизить с 30 до 13% https://www.klerk.ru/buh/news/534724/
Замороженные зарубежные активы россиян могут обменять на акции иностранных инвесторов https://www.klerk.ru/buh/news/535771/
Недружественные иностранцы могут попасть на российский рынок акций. Но с некоторыми условиями https://www.klerk.ru/buh/news/534734/


{'ндфл': 1530.9651348447717,
 'минфин': 1494.8984139909808,
 'бумаг': 1488.3781242500547,
 'взнос': 1454.768159050196,
 'комитет': 1368.2564585557532,
 'разъяснен': 1284.8614019980575,
 'форм': 1165.6115937377554,
 'бухгалтер': 1068.4364386756165,
 'уплат': 1005.1691380882477,
 'письм': 977.4662610824308}

[{'word': 'взносов', 'short': 'взнос', 'count': 0.9108627444555908},
 {'word': 'Минфин', 'short': 'минфин', 'count': 0.8208506648116617},
 {'word': 'бухгалтеров', 'short': 'бухгалтер', 'count': 0.7987927240211623},
 {'word': 'бумаге', 'short': 'бумаг', 'count': 0.7924907417044055},
 {'word': 'форме', 'short': 'форм', 'count': 0.7403175319179741},
 {'word': 'дохода', 'short': 'доход', 'count': 0.7326672405116571},
 {'word': 'НДФЛ', 'short': 'ндфл', 'count': 0.7316495372106746},
 {'word': 'это', 'short': 'эт', 'count': 0.7253073109435502},
 {'word': 'счете', 'short': 'счет', 'count': 0.7214786042034351},
 {'word': 'налогов', 'short': 'налог', 'count': 0.6981504124519681}]


cluster:  5  count= 17
Россияне скупают дешевую технику https://www.klerk.ru/buh/news/535068/
Страны бывшего СССР стали больше поставлять товаров Россию. С запада https://www.klerk.ru/buh/news/535112/
Apple представила iPhone 14. Его будут ввозить в Россию по параллельному импорту https://www.klerk.ru/buh/news/534737/


{'импорт': 408.1934878405194,
 'чек': 369.21647825341535,
 'рост': 292.68213719429986,
 'июн': 281.2518371629036,
 'поставок': 237.70851958833023,
 'товар': 219.72451321972233,
 'торговл': 184.04082081721197,
 'цен': 179.72386571706912,
 '2022': 159.5884371928986,
 'ввоз': 136.5262063859418}

...

{'кредитован': 791.7130601864578,
 'ставк': 680.8958225319487,
 'интерфакс': 640.6324182143316,
 'кредит': 457.12501934521106,
 'программ': 391.66331782479153,
 'ипотек': 295.8642068705149,
 'мал': 268.7172935447045,
 'сумм': 267.76573169370874,
 'млн': 198.4172223369055,
 'кред': 183.8931708416775}

[{'word': 'кредитования', 'short': 'кредитован', 'count': 3.306343144844237},
 {'word': 'Ставка', 'short': 'ставк', 'count': 3.011702357385753},
 {'word': 'Интерфакс', 'short': 'интерфакс', 'count': 2.943928302524843},
 {'word': 'кредита', 'short': 'кредит', 'count': 2.4871609354608473},
 {'word': 'программе', 'short': 'программ', 'count': 2.3259037331841137},
 {'word': 'ипотека', 'short': 'ипотек', 'count': 2.092368327726842},
 {'word': 'малый', 'short': 'мал', 'count': 1.9940069017002076},
 {'word': 'сумму', 'short': 'сумм', 'count': 1.9798342869050272},
 {'word': 'кредит', 'short': 'кред', 'count': 1.7035907074314631},
 {'word': 'млн', 'short': 'млн', 'count': 1.6799086210332228}]


cluster:  4  count= 12
Предприниматели будут проходить идентификацию на Wildberries через Госуслуги. Сейчас мошенники создают фейковые карточки товаров для развода https://www.klerk.ru/buh/news/535034/
Мошенники получают реквизиты банковских карт через аккаунты в «Яндекс» и VK https://www.klerk.ru/buh/news/535427/
Спектакль в двух действиях. Телефонные мошенники придумали новую схему https://www.klerk.ru/buh/news/535226/


{'мошенник': 1070.0582716537801,
 'жертв': 787.4620494965977,
 'якоб': 317.6402840477967,
 'звонок': 204.6184334914796,
 'полиц': 186.16493446024714,
 'мошенничеств': 184.61030081638265,
 'звонк': 184.3492940148883,
 'кибербезопасн': 182.29383150322406,
 'схем': 174.99315745616548,
 'идентификац': 166.96070788834783}

[{'word': 'мошенников', 'short': 'мошенник', 'count': 4.0891567764093635},
 {'word': 'жертвами', 'short': 'жертв', 'count': 3.622808273066313},
 {'word': 'якобы', 'short': 'якоб', 'count': 2.425374801758079},
 {'word': 'звонок', 'short': 'звонок', 'count': 2.064693509911183},
 {'word': 'полицию', 'short': 'полиц', 'count': 1.9693917930114246},
 {'word': 'мошенничества',
  'short': 'мошенничеств',
  'count': 1.9611516599862722},
 {'word': 'звонка', 'short': 'звонк', 'count': 1.9597783044820802},
 {'word': 'кибербезопасности',
  'short': 'кибербезопасн',
  'count': 1.948791460448544},
 {'word': 'идентификацию',
  'short': 'идентификац',
  'count': 1.8650329972971649},
 {'word': 'схема', 'short': 'схем', 'count': 1.8005517624526988}]